In [1]:
import torch

from model_analysis_folders.visual_networks.resnet50.build_network import build_net
from robustness.attacker import AttackerModel

attacker_model, _ = build_net()
attacker_model: AttackerModel = attacker_model.eval().cuda()

In [2]:
# create input data (reference_image)
inp = torch.randn(10, 3, 224, 224).cuda()

layer_to_invert = "layer4"
step_size = 0.5
iterations = 100

In [3]:
from robustness.custom_synthesis_losses import InversionLossLayer

synth_kwargs = {
    # same simple loss as in the paper
    'custom_loss': InversionLossLayer(layer_to_invert, normalize_loss=True),
    'constraint': '2',  # norm constraint. L2, L_inf, etc.
    'eps': 100000,  # why this high? this is weird, usually 8/255 or some is used
    'step_size': step_size,  # essentially works like learning rate. halved every 3000 iterations (default: 1.0)
    'iterations': iterations,  # iterations to generate one adv example
    'do_tqdm': False,
    'targeted': True,
    'use_best': False
}

In [4]:
with torch.no_grad():
    (reference_output, reference_representation, reference_activations), reference_image = attacker_model(
        inp.cuda(), with_latent=True, fake_relu=True)

inverted_reference_representation = reference_activations[layer_to_invert].contiguous().view(
    reference_activations[layer_to_invert].size(0), -1)

In [5]:
def run_steps():
    attacker_model(
        inp,
        inverted_reference_representation.clone(),
        make_adv=True,
        **synth_kwargs,
        with_latent=True,
        fake_relu=True
    )
    print("done")

In [6]:
%timeit -r 10 -n 1 run_steps()

done
done
done
done
done
done
done
done
done
done
6.05 s ± 113 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


count_parameters(attacker_model.model)

25557032